In [2]:
library("reticulate")
# use_python("C:/Users/berkc/Miniconda3/envs/R", required = T)
use_python("C:/Users/kapusub/Anaconda3/envs/deep_learning_PIML", required = T)
py_config()

python:         C:/Users/kapusub/Anaconda3/envs/deep_learning_PIML/python.exe
libpython:      C:/Users/kapusub/Anaconda3/envs/deep_learning_PIML/python37.dll
pythonhome:     C:\Users\kapusub\ANACON~1\envs\DEEP_L~2
version:        3.7.7 (default, May  6 2020, 11:45:54) [MSC v.1916 64 bit (AMD64)]
Architecture:   64bit
numpy:          C:\Users\kapusub\ANACON~1\envs\DEEP_L~2\lib\site-packages\numpy
numpy_version:  1.18.1

NOTE: Python version was forced by use_python function

In [3]:
source("sobolGP_poro_np.R")
source("sobolGP_poro_np_upd.R")
source("sobolGPmethods.R")
source("pickfreeze.R")

# Pre-trained model parameters

In [4]:
gp = source_python("GP_pre_train_AM.py")
params <- Gp_phy()
params

0.15**2 * RBF(length_scale=[3.17e+08, 1.21e+06])

In [21]:
numobs <- c(5,10,20,30,38)
case1 <- c("GP_loss_MC_p_AM")
# case1 <- c("GP_MC_p_AM")
case1 <- c()
case2 <- c("GP_upd_MC_p_AM","GP_upd_loss_MC_p_AM")
# case2 <- c()
d <- 2; n <- 2000
candidate <- data.frame(matrix(runif(d * 2), nrow = 2))
X1 <- data.frame(matrix(runif(d * n), nrow = n))
X2 <- data.frame(matrix(runif(d * n), nrow = n))

for(j in case1){
    source("sobolGP_poro_np.R")
    GP_file = paste(j, ".py", sep="")    
    cat(GP_file,"\n")
    for(i in numobs){
    
        start_time <- Sys.time()
        #--------------------------------------#
        # Kriging-based Sobol
        #--------------------------------------#
        res <- sobolGP_poro_np(
        model = "GP",
        type="UK",
        MCmethod="sobol2002",
        X1,
        X2,
        nsim = 100,
        conf = 0.95,
        nboot=1,
        sequential = FALSE,
        candidate,
        sequential.tot=FALSE,
        max_iter = 100,
        filename=GP_file,
        nobs = i
        )

        end_time <- Sys.time()
        print(end_time - start_time)
        
        # combine number of observation vector with Sobol indices
#         data <- cbind(i, res$S$mean)
        data <- cbind(i, res$S$mean, t(res$S$ci[1,]), t(res$S$ci[2,]))
        dat <- as.data.frame(data)
        
        # total index
#         dataT <- cbind(i, res$T$mean)
        dataT <- cbind(i, res$T$mean, t(res$T$ci[1,]), t(res$T$ci[2,]))
        datT <- as.data.frame(dataT)
        
        # write to file
        results_dir <- "../../../../results/AM/"
        # filename <- paste(results_dir, "Case1_", toString(num_obs), ".csv", sep = "")
        # write.csv(x=dat, file=filename)
        # write.table(dat, file=filename, append=T)
        filename <- paste(results_dir, paste(j, "_first_", n,"_10restart.csv", sep=""), sep = "")
        write.table(dat, filename, sep = ",", col.names = !file.exists(filename), row.names = F, append = T)
        
        filenameT <- paste(results_dir, paste(j, "_tot_", n,"_10restart.csv", sep=""), sep = "")
        write.table(datT, filenameT, sep = ",", col.names = !file.exists(filenameT), row.names = F, append = T)
    }
}






for(j in case2){
    source("sobolGP_poro_np_upd.R")
    GP_file = paste(j, ".py", sep="")    
    cat(GP_file,"\n")
    for(i in numobs){
    
        start_time <- Sys.time()
        #--------------------------------------#
        # Kriging-based Sobol
        #--------------------------------------#
        res <- sobolGP_poro_np_upd(
        model = "GP",
        type="UK",
        MCmethod="sobol2002",
        X1,
        X2,
        nsim = 100,
        conf = 0.95,
        nboot=1,
        sequential = FALSE,
        candidate,
        sequential.tot=FALSE,
        max_iter = 100,
        filename=GP_file,
        params,
        nobs = i
        )
        
        end_time <- Sys.time()
        print(end_time - start_time)
        
        # combine number of observation vector with Sobol indices
        data <- cbind(i, res$S$mean, t(res$S$ci[1,]), t(res$S$ci[2,]))
        dat <- as.data.frame(data)
        
        # total index
        dataT <- cbind(i, res$T$mean, t(res$T$ci[1,]), t(res$T$ci[2,]))
        datT <- as.data.frame(dataT)
        
        # write to file
        results_dir <- "../../../../results/AM/"
        filename <- paste(results_dir, paste(j, "_first_", n,"_10restart.csv", sep=""), sep = "")
        write.table(dat, filename, sep = ",", col.names = !file.exists(filename), row.names = F, append = T)
        
        filenameT <- paste(results_dir, paste(j, "_tot_", n,"_10restart.csv", sep=""), sep = "")
        write.table(datT, filenameT, sep = ",", col.names = !file.exists(filenameT), row.names = F, append = T)
    }
}

GP_upd_MC_p_AM.py 
Time difference of 32.588 secs


Warning message in write.table(dat, filename, sep = ",", col.names = !file.exists(filename), :
"appending column names to file"Warning message in write.table(datT, filenameT, sep = ",", col.names = !file.exists(filenameT), :
"appending column names to file"

Time difference of 34.9862 secs
Time difference of 35.28511 secs
Time difference of 36.01845 secs
Time difference of 36.06416 secs
GP_upd_loss_MC_p_AM.py 
Time difference of 35.37222 secs


Warning message in write.table(dat, filename, sep = ",", col.names = !file.exists(filename), :
"appending column names to file"Warning message in write.table(datT, filenameT, sep = ",", col.names = !file.exists(filenameT), :
"appending column names to file"

Time difference of 35.37578 secs
Time difference of 35.51143 secs
Time difference of 36.75625 secs
Time difference of 36.83154 secs


In [13]:
res

ERROR while rich displaying an object: Error in cat("\nModel runs:", sGP$call$model@n, "\n"): trying to get slot "n" from an object of a basic class ("character") with no slots

Traceback:
1. FUN(X[[i]], ...)
2. tryCatch(withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler), error = outer_handler)
3. tryCatchList(expr, classes, parentenv, handlers)
4. tryCatchOne(expr, names, parentenv, handlers[[1L]])
5. doTryCatch(return(expr), name, parentenv, handler)
6. withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error 